# Stata Admin Programming

In [1]:
import stata_setup
stata_setup.config("C:/Program Files/Stata17", "se")

SyntaxError: invalid syntax (<ipython-input-1-6e2665b4b1c9>, line 1)

## Set up simulated data in Stata
Data broadly resembles a year panel based on IAB data with person (persnr), establishment (betnr) and year identifiers.

In [ ]:
%%stata
clear all
set obs 10000000

gen betnr = runiformint(1, 1000)
gen persnr = runiformint(1,100000)
gen year = runiformint(1972, 2019)

gen double tentgelt = runiform(0, 100000) 

gen german = runiformint(0,1)
bysort persnr: replace german = german[_N]

# Working with yearly panels
## Egen: max()
    - initialize variable
    - use max() function to get largest value into last observation determined by 'bysort'
        - note missings are ignored by max() unless *all* observations are missing
    - replace within 'bysort' by the last observation

In [ ]:
%%stata
gen max_tentgelt = tentgelt
bysort persnr: replace max_tentgelt = max(max_tentgelt[_n-1], max_tentgelt)
bysort persnr: replace max_tentgelt = max_tentgelt[_N]

### Run-time comparison


In [ ]:
%%stata
drop max_tentgelt
timer on 1
gen double max_tentgelt = tentgelt
bysort persnr: replace max_tentgelt = max(max_tentgelt[_n-1], max_tentgelt)
bysort persnr: replace max_tentgelt = max_tentgelt[_N]
timer off 1

timer on 2
egen double max_tentgelt_egen = max(tentgelt), by(persnr)
timer off 2

timer on 3
gegen double max_tentgelt_gegen = max(tentgelt), by(persnr)
timer off 3

assert max_tentgelt == max_tentgelt_egen
assert max_tentgelt == max_tentgelt_gegen

timer list